In [1]:
from __future__ import print_function, division
#virtualenv -p python3 venv

In [2]:
%matplotlib inline
import matplotlib.pyplot as plt
plt.style.use('ggplot')
import cPickle as pk
import pandas as pd
import numpy as np
from IPython.display import clear_output

In [3]:
import os
import sys
base_module_path = os.path.abspath(os.path.join('..'))
if base_module_path not in sys.path:
    sys.path.append(base_module_path)
import cervix as c

Using TensorFlow backend.


In [4]:
ls ../data/train/

Type_1/ Type_2/ Type_3/


In [5]:
df = c.data.make_base_df()
df = c.processing.transform_save_imgs(df, c.processing.resize_n, n=299);

In [6]:
train, validate, test = c.data.split_df(df); 

In [7]:
print(train.shape)
train.head()

(888, 5)


,path,Type,filetype,num_id,processed_path
137,../data/train/Type_1/441.jpg,1,jpg,441,../data/processed/resize_299/train/441.jpg
130,../data/train/Type_1/396.jpg,1,jpg,396,../data/processed/resize_299/train/396.jpg
1032,../data/train/Type_3/1003.jpg,3,jpg,1003,../data/processed/resize_299/train/1003.jpg
177,../data/train/Type_1/623.jpg,1,jpg,623,../data/processed/resize_299/train/623.jpg
1303,../data/train/Type_3/480.jpg,3,jpg,480,../data/processed/resize_299/train/480.jpg


In [8]:
print(validate.shape)
validate.head()

(296, 5)


,path,Type,filetype,num_id,processed_path
1126,../data/train/Type_3/1300.jpg,3,jpg,1300,../data/processed/resize_299/train/1300.jpg
1446,../data/train/Type_3/897.jpg,3,jpg,897,../data/processed/resize_299/train/897.jpg
588,../data/train/Type_2/225.jpg,2,jpg,225,../data/processed/resize_299/train/225.jpg
907,../data/train/Type_2/785.jpg,2,jpg,785,../data/processed/resize_299/train/785.jpg
674,../data/train/Type_2/381.jpg,2,jpg,381,../data/processed/resize_299/train/381.jpg


In [9]:
print(test.shape)
test.head()

(297, 5)


,path,Type,filetype,num_id,processed_path
1067,../data/train/Type_3/1107.jpg,3,jpg,1107,../data/processed/resize_299/train/1107.jpg
1305,../data/train/Type_3/49.jpg,3,jpg,49,../data/processed/resize_299/train/49.jpg
328,../data/train/Type_2/1138.jpg,2,jpg,1138,../data/processed/resize_299/train/1138.jpg
704,../data/train/Type_2/428.jpg,2,jpg,428,../data/processed/resize_299/train/428.jpg
191,../data/train/Type_1/7.jpg,1,jpg,7,../data/processed/resize_299/train/7.jpg


# Model creation

In [10]:
from keras.applications.inception_v3 import InceptionV3
from keras.preprocessing import image
from keras.models import Model
from keras.layers import Dense, GlobalAveragePooling2D
from keras import backend as K

# create the base pre-trained model
base_model = InceptionV3(weights='imagenet', include_top=False)

# add a global spatial average pooling layer
x = base_model.output
x = GlobalAveragePooling2D()(x)
# let's add a fully-connected layer
x = Dense(1024, activation='relu')(x)
# and a logistic layer -- let's say we have 200 classes
    # we have three classes so usung 3 on dense predictions
predictions = Dense(3, activation='softmax')(x)

# this is the model we will train
model = Model(inputs=base_model.input, outputs=predictions)

# we chose to train the top 2 inception blocks, i.e. we will freeze
# the first 172 layers and unfreeze the rest:
for layer in model.layers[:172]:
    layer.trainable = False
for layer in model.layers[172:]:
    layer.trainable = True

In [11]:
# compile the model (should be done *after* setting layers to non-trainable)
# TODO: loss function should be log loss. 'categorical_crossentropy' / c.submission.keras_log_loss
model.compile(optimizer='rmsprop', loss='categorical_crossentropy')

In [12]:
# let's visualize layer names and layer indices to see how many layers
# we should freeze:
for i, layer in enumerate(base_model.layers):
    print(i, layer.name, layer.trainable)

0 input_1 False
1 conv2d_1 False
2 batch_normalization_1 False
3 activation_1 False
4 conv2d_2 False
5 batch_normalization_2 False
6 activation_2 False
7 conv2d_3 False
8 batch_normalization_3 False
9 activation_3 False
10 max_pooling2d_1 False
11 conv2d_4 False
12 batch_normalization_4 False
13 activation_4 False
14 conv2d_5 False
15 batch_normalization_5 False
16 activation_5 False
17 max_pooling2d_2 False
18 conv2d_9 False
19 batch_normalization_9 False
20 activation_9 False
21 conv2d_7 False
22 conv2d_10 False
23 batch_normalization_7 False
24 batch_normalization_10 False
25 activation_7 False
26 activation_10 False
27 average_pooling2d_1 False
28 conv2d_6 False
29 conv2d_8 False
30 conv2d_11 False
31 conv2d_12 False
32 batch_normalization_6 False
33 batch_normalization_8 False
34 batch_normalization_11 False
35 batch_normalization_12 False
36 activation_6 False
37 activation_8 False
38 activation_11 False
39 activation_12 False
40 mixed0 False
41 conv2d_16 False
42 batch_normaliza

In [13]:
# train the model on the new data for a few epochs
batch_size = 20
model.fit_generator(c.processing.df_to_keras_generator(train, batch_size, grayscale=False), 
                    steps_per_epoch=5,
                    epochs=9)


Epoch 1/9
5/5 [==============================] - 51s - loss: 3.5197     
Epoch 2/9
5/5 [==============================] - 44s - loss: 1.8390    
Epoch 3/9
5/5 [==============================] - 37s - loss: 2.0230    
Epoch 4/9
5/5 [==============================] - 42s - loss: 1.4050    
Epoch 5/9
5/5 [==============================] - 42s - loss: 1.0466    
Epoch 6/9
5/5 [==============================] - 40s - loss: 1.2396    
Epoch 7/9
5/5 [==============================] - 43s - loss: 1.1430    
Epoch 8/9
5/5 [==============================] - 45s - loss: 1.0951    
Epoch 9/9
5/5 [==============================] - 41s - loss: 1.0412    


In [ ]:
batch_size_t = 10
model.evaluate_generator(c.processing.df_to_keras_generator(test, batch_size_t, grayscale=False)
                    ,int(len(test)/batch_size_t))

In [14]:
batch_size_t = 10
predictions = model.predict_generator(c.processing.df_to_keras_generator(test, 1, grayscale=False)
                    ,len(test))
predictions

array([[  7.31924354e-19,   1.00000000e+00,   7.33644644e-31],
       [  2.00168848e-01,   6.41657650e-01,   1.58173516e-01],
       [  2.84936650e-05,   9.99971509e-01,   1.15600564e-10],
       [  5.75042143e-02,   9.39778149e-01,   2.71764514e-03],
       [  1.46036938e-01,   7.35258698e-01,   1.18704304e-01],
       [  8.01805465e-04,   9.99198139e-01,   7.04809580e-08],
       [  1.74474001e-01,   6.42671525e-01,   1.82854429e-01],
       [  6.01895377e-02,   4.53637928e-01,   4.86172557e-01],
       [  0.00000000e+00,   1.00000000e+00,   0.00000000e+00],
       [  1.18267059e-01,   8.31826866e-01,   4.99061346e-02],
       [  9.13271531e-02,   4.43406582e-01,   4.65266258e-01],
       [  1.18528076e-10,   1.00000000e+00,   4.58197604e-20],
       [  9.84571204e-02,   8.79329860e-01,   2.22130064e-02],
       [  5.85705125e-16,   1.00000000e+00,   4.76242085e-31],
       [  7.57704228e-02,   9.10969019e-01,   1.32605573e-02],
       [  1.35799766e-01,   6.98133886e-01,   1.6606634

In [15]:
predictions.sum(0)

array([  20.18827629,  251.36013794,   25.45156479], dtype=float32)

In [16]:
test.Type.value_counts()

2    156
3     93
1     48
Name: Type, dtype: int64

In [17]:
print(len(predictions), len(test))

297 297


In [ ]:
ldf.sort_values('log_l', ascending=False).path.iloc[0]

In [ ]:
c.data.check_image(ldf.sort_values('log_l', ascending=False).path.iloc[0])

In [ ]:
c.data.check_image(ldf.sort_values('log_l', ascending=False).resize_100_path.iloc[0])

In [ ]:
c.data.check_image(ldf.sort_values('log_l', ascending=False).path.iloc[1])

In [ ]:
c.data.check_image(ldf.sort_values('log_l', ascending=False).resize_100_path.iloc[1])

In [ ]:
c.data.check_image(ldf.sort_values('log_l', ascending=False).path.iloc[3])

In [ ]:
c.data.check_image(ldf.sort_values('log_l', ascending=False).resize_100_path.iloc[3])